In [1]:
import tensorflow as tf

In [2]:

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import drive
drive.mount("/content/gdrive") 

Mounted at /content/gdrive


In [4]:
# %mkdir gdrive/MyDrive/DSKU/datasets
#%cd gdrive/MyDrive/Datasets
%cd gdrive/MyDrive/DSKU/datasets

/content/gdrive/MyDrive/DSKU/datasets


In [5]:
# load all the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [6]:
# focused countries

COUNTRIES = """Argentina
Bolivia
Brazil
Chile
Colombia
Costa Rica
Cuba
Dominican Republic
Ecuador
El Salvador
French Guiana
Guadeloupe
Guatemala
Haiti
Honduras
Martinique
Mexico
Nicaragua
Panama
Paraguay
Peru
Puerto Rico
Saint Barthélemy
Saint Martin
Uruguay
Venezuela
Brunei
Cambodia
Timor-Leste
Indonesia
Laos
Malaysia
Burma (Myanmar)
Philippines
Singapore
Thailand
Vietnam""".split("\n")

In [7]:
def readCsvFile(filepath):
    """
        Function to read csv file
        args:
            filepath : path to csv file
        return:
            df : dataframe
    """
    
    df = pd.read_csv(filepath)
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    
    df.drop(df.columns[-1], inplace=True, axis = 1)
    
    return df

In [8]:
%pwd

'/content/gdrive/MyDrive/DSKU/datasets'

In [9]:
raw_us_aid = readCsvFile('us_foreign_aid_complete.csv')

In [10]:
raw_us_aid.head(2)

,country_id,country_code,country_name,region_id,region_name,income_group_id,income_group_name,income_group_acronym,managing_agency_id,managing_agency_acronym,managing_agency_name,managing_sub-agency_or_bureau_id,managing_sub-agency_or_bureau_acronym,managing_sub-agency_or_bureau_name,implementing_partner_category_id,implementing_partner_category_name,implementing_partner_sub-category_id,implementing_partner_sub-category_name,implementing_partner_id,implementing_partner_name,international_category_id,international_category_name,international_sector_code,international_sector_name,international_purpose_code,international_purpose_name,us_category_id,us_category_name,us_sector_id,us_sector_name,funding_account_id,funding_account_name,funding_agency_id,funding_agency_name,funding_agency_acronym,foreign_assistance_objective_id,foreign_assistance_objective_name,aid_type_group_id,aid_type_group_name,aid_type_id,aid_type_name,activity_id,submission_id,activity_name,activity_description,activity_project_number,activity_budget_amount,activity_start_date,activity_end_date,transaction_type_id,transaction_type_name,fiscal_year,transaction_date,current_dollar_amount,constant_dollar_amount,submission_activity_id,finance_type,dac_flow_id
0,4,AFG,Afghanistan,4,South and Central Asia,1.0,Low Income Country,LIC,1,USAID,U.S. Agency for International Development,20,ASIA,Bureau for Asia,1,Government,1,Government - United States,1000001,U.S. Government - U.S. Agency for Internationa...,1,Education,112,Basic Education,11220,Primary education,4,Education and Social Services,20,Basic Education,72x1021,"Agency for International Development, Developm...",1,U.S. Agency for International Development,USAID,1,Economic,4,Technical Assistance,13,Technical Cooperation - Other,144477,76,Office Of Social Sector Dev. - Education,Office Of Social Sector Dev. - Education,306-002,NaN,NaN,NaN,2,Obligations,2006,01MAR2006,37760,47793,30831,110,1
1,4,AFG,Afghanistan,4,South and Central Asia,1.0,Low Income Country,LIC,1,USAID,U.S. Agency for International Development,20,ASIA,Bureau for Asia,1,Government,1,Government - United States,1000001,U.S. Government - U.S. Agency for Internationa...,1,Education,112,Basic Education,11220,Primary education,4,Education and Social Services,20,Basic Education,72x1021,"Agency for International Development, Developm...",1,U.S. Agency for International Development,USAID,1,Economic,4,Technical Assistance,13,Technical Cooperation - Other,144477,76,Office Of Social Sector Dev. - Education,Office Of Social Sector Dev. - Education,306-002,NaN,NaN,NaN,3,Disbursements,2006,01MAR2006,983,1244,30831,110,1


In [11]:
raw_us_aid['fiscal_year'] = np.where(raw_us_aid.fiscal_year=="1976tq", "1976", raw_us_aid.fiscal_year)

In [12]:
raw_us_aid['fiscal_year'] = raw_us_aid['fiscal_year'].astype(int)

In [13]:
# consider latest government
latest_foreign_aid = raw_us_aid[(raw_us_aid.fiscal_year>=2021) & (raw_us_aid.fiscal_year<2022)]\
                  .query("country_name == @COUNTRIES")

In [14]:
latest_foreign_aid.groupby(['country_code']).agg({'activity_description':'nunique'}).reset_index()

,country_code,activity_description
0,ARG,101
1,BOL,32
2,BRA,150
3,CHL,16
4,COL,1972
5,CRI,1051
6,CUB,18
7,DOM,302
8,ECU,452
9,GTM,2017


In [15]:
latest_foreign_aid[latest_foreign_aid.duplicated(['country_code', 'activity_description'])].head()

,country_id,country_code,country_name,region_id,region_name,income_group_id,income_group_name,income_group_acronym,managing_agency_id,managing_agency_acronym,managing_agency_name,managing_sub-agency_or_bureau_id,managing_sub-agency_or_bureau_acronym,managing_sub-agency_or_bureau_name,implementing_partner_category_id,implementing_partner_category_name,implementing_partner_sub-category_id,implementing_partner_sub-category_name,implementing_partner_id,implementing_partner_name,international_category_id,international_category_name,international_sector_code,international_sector_name,international_purpose_code,international_purpose_name,us_category_id,us_category_name,us_sector_id,us_sector_name,funding_account_id,funding_account_name,funding_agency_id,funding_agency_name,funding_agency_acronym,foreign_assistance_objective_id,foreign_assistance_objective_name,aid_type_group_id,aid_type_group_name,aid_type_id,aid_type_name,activity_id,submission_id,activity_name,activity_description,activity_project_number,activity_budget_amount,activity_start_date,activity_end_date,transaction_type_id,transaction_type_name,fiscal_year,transaction_date,current_dollar_amount,constant_dollar_amount,submission_activity_id,finance_type,dac_flow_id
107781,32,ARG,Argentina,6,Western Hemisphere,3.0,Upper Middle Income Country,UMIC,2,STATE,Department of State,62,INL,International Narcotics and Law Enforcement Af...,7,Multilateral,12,Multilateral - United Nations,4000455,United Nations Office on Drugs and Crime,3,Governance,151,Government and Civil Society,15130,Legal and judicial development,2,"Democracy, Human Rights, and Governance",7,Rule of Law and Human Rights,11x1022,International Narcotics Control and Law Enforc...,2,Department of State,STATE,1,Economic,3,Project-Type,8,Project-type interventions - not Investment Re...,345875,105,Bureau of International Narcotics and Law Enfo...,U.S. Department of State: Bureau of Internatio...,SINLEC21LA3353,NaN,NaN,2021-09-30,3,Disbursements,2021,30SEP2021,600000,589998,134457,110,1
107783,32,ARG,Argentina,6,Western Hemisphere,3.0,Upper Middle Income Country,UMIC,2,STATE,Department of State,62,INL,International Narcotics and Law Enforcement Af...,8,Enterprises,19,Enterprises - United States,3990008,Enterprise - United States Other,10,Administrative Costs,911,Operating Expenses,91010,Operating Expenses,8,Program Support,41,Direct Administrative Costs,11x1022,International Narcotics Control and Law Enforc...,2,Department of State,STATE,1,Economic,6,Administrative Costs,17,Administrative costs not included elsewhere,312736,105,Bureau of International Narcotics and Law Enfo...,U.S. Department of State: Bureau of Internatio...,20730110299,NaN,NaN,2020-12-02,2,Obligations,2021,02DEC2020,13440,13216,101318,110,1
107784,32,ARG,Argentina,6,Western Hemisphere,3.0,Upper Middle Income Country,UMIC,2,STATE,Department of State,62,INL,International Narcotics and Law Enforcement Af...,8,Enterprises,19,Enterprises - United States,3990008,Enterprise - United States Other,10,Administrative Costs,911,Operating Expenses,91010,Operating Expenses,8,Program Support,41,Direct Administrative Costs,11x1022,International Narcotics Control and Law Enforc...,2,Department of State,STATE,1,Economic,6,Administrative Costs,17,Administrative costs not included elsewhere,312739,105,Bureau of International Narcotics and Law Enfo...,U.S. Department of State: Bureau of Internatio...,20730110299,NaN,NaN,2021-05-04,2,Obligations,2021,04MAY2021,-900,-885,101321,110,1
107785,32,ARG,Argentina,6,Western Hemisphere,3.0,Upper Middle Income Country,UMIC,2,STATE,Department of State,62,INL,International Narcotics and Law Enforcement Af...,8,Enterprises,19,Enterprises - United States,3990008,Enterprise - United States Other,10,Administrative Costs,911,Operating Expenses,91010,Operating Expenses,8,Program Support,41,Direct Administrative Costs,11x1022,International Narcotics Control and Law Enforc...,2,Department of State,STATE,1,Economic,6,Administrative Costs,17,Ad

In [16]:
latest_foreign_aid['region_name'].value_counts()

Western Hemisphere       53778
East Asia and Oceania    17930
Name: region_name, dtype: int64

# Text analytics
## Clustering activity description using topic modelling
## Approach 2: contextulized topic modeling 

In [17]:
%%capture
!pip install contextualized_topic_models
!pip install sentence-transformers

In [18]:
from contextualized_topic_models.models.ctm import ZeroShotTM, CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk
import torch
import random
import numpy as np

In [19]:

def fix_seeds():
  torch.manual_seed(10)
  torch.cuda.manual_seed(10)
  np.random.seed(10)
  random.seed(10)
  torch.backends.cudnn.enabled = False
  torch.backends.cudnn.deterministic = True

In [20]:
from nltk.corpus import stopwords as stop_words
nltk.download('stopwords')
stopwords = list(set(stop_words.words('english')))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
latest_foreign_aid['activity_description'] = latest_foreign_aid['activity_description'].astype(str)

In [22]:
activity_df = latest_foreign_aid.groupby("country_code")['activity_description'].apply(' '.join).reset_index()

In [23]:
activity_df.head()

,country_code,activity_description
0,ARG,"U.S. Department of State: Bureau of Democracy,..."
1,BOL,USAID redacted this field in accordance with t...
2,BRA,USAID redacted this field in accordance with t...
3,CHL,Natural Hazard Mitigation Global Technical Ass...
4,COL,USAID redacted this field in accordance with t...


In [24]:
activity_df.shape

(30, 2)

In [25]:

activity_df.activity_description.apply(lambda x:len(x))

0      216460
1       36221
2      364251
3       35149
4     2473130
5     1250301
6       53761
7      639143
8      832511
9     2493855
10    1787331
11     789080
12    1114818
13     607652
14     320472
15    1752786
16     357422
17     123541
18     199043
19     918896
20    1531719
21     984048
22     423214
23       1077
24    1522151
25     589827
26     181767
27      10148
28     184925
29     888647
Name: activity_description, dtype: int64

In [26]:
# get text
activities = activity_df['activity_description'].tolist()

In [27]:
uniq_activities = set(activities)

In [28]:
documents = list(uniq_activities)


In [29]:

len(documents)

30

In [30]:
documents[1]

"Health: PAHO Umbrella Grant 2016-2021 Housed in a 19th-century villa, the Blanes Municipal Museum displays Uruguayan art from the nation's founding in the 19th century to the present day. This project includes the conservation of the building's masonry facades. Housed in a 19th-century villa, the Blanes Municipal Museum displays Uruguayan art from the nation's founding in the 19th century to the present day. This project includes the conservation of the building's masonry facades. The eastern seaboard of Uruguay includes a chain of 5 coastal lagoons, supporting globally important populations of migratory shorebirds that use the surrounding grasslands, wetlands and sandy areas, as well as the lagoon shores, as wintering areas. This region, covering 2,471,000 acres, is under pressure by threats such as agriculture, intensive livestock ranching, urbanization and recreational activities such as uncontrolled tourism. Partners will secure long-term shorebird conservation by developing and e

In [31]:
documents[0]

Output hidden; open in https://colab.research.google.com to view.

In [32]:

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)


In [33]:
preprocessed_documents, unpreprocessed_corpus, vocab, _ = sp.preprocess()

In [34]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v2")

In [35]:
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
corpus = [d.split() for d in preprocessed_documents]

num_topics = [5, 10, 15, 20]
num_runs = 5

best_topic_coherence = -999
best_num_topics = 0
for n_components in num_topics:
  for i in range(num_runs):
    print("num topics:", n_components, "/ num run:", i)
    ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, 
                     n_components=n_components, num_epochs=50)
    ctm.fit(training_dataset) # run the model
    coh = CoherenceNPMI(ctm.get_topic_lists(10), corpus)
    coh_score = coh.score()
    print("coherence score:", coh_score)
    if best_topic_coherence < coh_score:
      best_topic_coherence = coh_score
      best_num_topics = n_components
    print("current best coherence", best_topic_coherence, "/ best num topics", best_num_topics)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


num topics: 5 / num run: 0


0it [00:00, ?it/s]

ZeroDivisionError: ignored

In [37]:
fix_seeds() # uncomment if you don't want to fix the random seeds

num_topics = 5


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=num_topics, num_epochs=50)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
ctm.fit(training_dataset) # run the model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
0it [00:00, ?it/s]

ZeroDivisionError: ignored

In [40]:
ctm.get_topic_lists(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: ignored

In [41]:
topics_predictions = ctm.training_doc_topic_distributions # get all the topic predictions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
np.sum(topics_predictions[1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: ignored

0it [00:11, ?it/s]


In [ ]:
np.sum(topics_predictions, axis=1)

In [ ]:
len(topics_predictions)

In [ ]:
len(training_dataset)

# Topic clustering

In [43]:
proba_dist = pd.read_pickle("results/proba_dist_topic_modelling.pkl")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FileNotFoundError: ignored

In [44]:
proba_dist.head(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: ignored

In [ ]:
# # create dataframe from topic distribution
# proba_dist = pd.DataFrame(topics_predictions)
# proba_dist['country_code'] = activity_df['country_code']

In [ ]:
%pwd

In [ ]:
# proba_dist.to_pickle("results/proba_dist_topic_modelling.pkl")

In [ ]:
activity_df.head(2)

In [ ]:
latest_foreign_aid.head(2)

In [ ]:
latest_foreign_aid['managing_agency_name'].unique()

In [ ]:
latest_foreign_aid.groupby(['country_code', 'managing_agency_name',
                            'transaction_type_name']).agg({'constant_dollar_amount':'sum'}).reset_index()

## KMeans

In [59]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from yellowbrick.cluster import silhouette_visualizer, SilhouetteVisualizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/yellowbrick/style/colors.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  mpl_ge_150 = LooseVersion(mpl.__version__) >= "1.5.0"
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/usr/local/lib/python3.10/dist-packages/yellowbrick/style/rcmod.py:31: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  mpl_ge_150 = LooseVersion(mpl.__version__) >= "1.5.0"
/us

In [60]:
# filter for only focused countries
proba_dist = proba_dist[proba_dist.country_code.isin(latest_foreign_aid.country_code)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: ignored

In [ ]:
# split data
X = proba_dist.iloc[:,0:-1]

In [ ]:
X.head(2)

In [ ]:

# fill na (since value not available, no Aid was provided for that country that year)
X.fillna(value=0, inplace=True)


In [ ]:
scaler = StandardScaler()


In [ ]:

scaled_x = scaler.fit_transform(X)


In [ ]:
silhouette_visualizer(KMeans(10, random_state=42), X, colors='yellowbrick')

In [ ]:
kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "random_state": 42
}

In [ ]:
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)

plt.figure(figsize=(6, 4))
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title("SSE for KMeans cluster range")



In [ ]:
n = 3

In [ ]:
country_3_clusters = KMeans(n_clusters=n, random_state=42)

In [ ]:
proba_dist['kmeans_3'] = country_3_clusters.fit_predict(X)


In [ ]:
centers = country_3_clusters.cluster_centers_


In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(scaled_x[:, 0], scaled_x[:, 1],
            c=proba_dist['kmeans_3'], s=50, cmap='viridis')

In [ ]:
proba_dist.groupby('kmeans_3').agg({'country_code':['unique', 'nunique']}).reset_index()

In [ ]:
print("# of Countries in cluster distribution")
proba_dist['kmeans_3'].value_counts()

In [ ]:
# mapping dictionary
country_mapping = dict(zip(latest_foreign_aid.country_code, latest_foreign_aid.country_name))

In [ ]:
proba_dist['country_name'] = proba_dist['country_code'].map(country_mapping)

In [ ]:
proba_dist['region'] = np.where(proba_dist.country_name.isin(COUNTRIES[26:-1]), "South East Asia", "Latin America")

In [ ]:
proba_dist.groupby(['kmeans_3', 'country_name']).agg({'country_code':"unique", "region":'unique'}).reset_index()

# Sentence Transformers
## Another approach

In [ ]:
%%capture
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer,util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
len(documents)

In [ ]:
#Encode all sentences
embeddings = model.encode(documents, normalize_embeddings=True)

In [ ]:
#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

In [ ]:
cos_sim[:3]

In [ ]:

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append((cos_sim[i][j], i, j))
# all_sentence_combinations  

In [ ]:
#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-2 most similar pairs:")
for score, i, j in all_sentence_combinations[0:2]:
    print("{} \t {} \t {:.4f}".format(documents[i], documents[j], cos_sim[i][j]))

In [ ]:
embedder = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
corpus_embeddings = embedder.encode(documents)

In [ ]:
len(corpus_embeddings[0])

In [ ]:
# Normalize the embeddings to unit length
# corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

In [ ]:
len(corpus_embeddings)

In [ ]:
X = scaler.fit_transform(corpus_embeddings)

In [ ]:
kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "random_state": 42
}

In [ ]:
sse = []
for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)

plt.figure(figsize=(6, 4))
plt.plot(range(1, 15), sse)
plt.xticks(range(1, 15))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title("SSE for KMeans cluster range")



In [ ]:
silhouette_visualizer(KMeans(10, random_state=42), corpus_embeddings, colors='yellowbrick')

In [ ]:

clustering_model = KMeans(n_clusters=4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)

In [ ]:
proba_dist['kmeans_word_embdd'] = cluster_assignment

In [ ]:
proba_dist.groupby(['kmeans_word_embdd']).agg({'country_code':"unique", "region":'unique'}).reset_index()

In [ ]:
centers = clustering_model.cluster_centers_


In [ ]:
corpus_embeddings[:,0]

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(corpus_embeddings[:, 2], corpus_embeddings[:, 3],
            c=proba_dist['kmeans_word_embdd'], s=50, cmap='viridis')

## DBSCAN clustering
### clustering using word embeddings (density based clustering)